Use with `vscode` and its `Data Wrangler`

In [ ]:
import re
import numpy as np
import torch
import pickle
_MAX_DEPTH=4
_MAX_KEYS = 100
_KV = []
_GLOBAL = {}
def load_pt(file):
    data = torch.load(file)
    return data

def load_pkl(file):
    with open(file, 'rb') as f:
        data = pickle.load(f)
    return data

def load_np(file):
    return np.load(file, allow_pickle=True)
    
def load(file):
    if file.endswith('.pt'):
        data = load_pt(file)
    elif file.endswith('.pkl'):
        data = load_pkl(file)
    elif file.endswith('.npy') or file.endswith('.npz'):
        data = load_np(file)
    print(f'load as {Type(data)} from {file}')
    return data

def Type(v): return str(type(v))[8:-2]
def is_dict(v): return hasattr(v,'keys')
def is_list(v): return hasattr(v,'__iter__') and not is_dict(v)
def expand_dict(data,prefix='',depth=0):
    indent = '    ' * depth
    if depth > _MAX_DEPTH:
        print(f'too deep, depth: {depth}')
        return
    _KV.append((indent,'',f'• {Type(data)}={len(data)}, depth={depth}'))
    depth += 1

    if is_list(data):
        for i,x in enumerate(data):
            expand_dict(x, f'{prefix}列{i}一',depth)
    elif is_dict(data):
        for k, v in data.items():
            # 如果k有非下划线符号，全部替换成_
            k = re.sub(r'[^a-zA-Z0-9_]', '_', k)

            if is_dict(v):
                _KV.append((indent,k,''))

            shape = None
            if is_dict(v):
                expand_dict(v,f'{prefix}{k}一',depth)
            else:
                if isinstance(v, torch.Tensor):
                    shape = v.shape
                    for i,x in enumerate(v.shape):
                        if x == 1:
                            v = v.squeeze(0)
                elif isinstance(v, np.ndarray):
                    shape = v.shape
                    for i,x in enumerate(v.shape):
                        if x == 1:
                            v=np.squeeze(v,0)
                if len(_GLOBAL.keys()) >= _MAX_KEYS:
                    print(f'too many keys, keys.len: {len(_GLOBAL.keys())}')
                    return
                _GLOBAL[f'{prefix}{k}'] = v

            v_str = ''
            if isinstance(v,np.ndarray):
                if shape != v.shape:
                    shape = f'{shape} → {v.shape}'
                v_str = f'{shape}^{v.dtype}'
            elif not is_dict(v):
                v_str = str(v)
            if not is_dict(v):
                _KV.append((indent,k,f'{v_str}\t{Type(v)}'))

def get_global(data):
    global _KV
    expand_dict(data)
    globals().update(_GLOBAL)
    max_name_len = 1
    for _,k,_ in _KV:
        if len(k) > max_name_len:
            max_name_len = len(k)
    for i,k,v in _KV:
        if k:
            k = f'{k:<{max_name_len}}'
        print(f'{i}{k} {v}')
    _KV = []

In [31]:
# _MAX_KEYS = 5
_FILE = [
    '/home/n/document/code/mocap/output/0_input_video/0_input_video.mp4_hand00.pt',
    '/home/n/document/code/mocap/output/hands_easy✋/hands_easy✋.pt',
    '/home/n/document/code/mocap/output/背越式跳高（慢动作）/mocap_背越式跳高（慢动作）.npz'
]
for _f in _FILE:
    get_global(load(_f))
print(list(_GLOBAL.keys()))

load as list from /home/n/document/code/mocap/output/0_input_video/0_input_video.mp4_hand00.pt
 • list=2, depth=0
     • dict=3, depth=1
    hand_bbox           [320.0, 392.0, 351.0, 433.0]	list
    is_right            1.0	float
    wilor_preds         
         • dict=9, depth=2
        global_orient       (1, 1, 3) → (3,)^float32	numpy.ndarray
        hand_pose           (1, 15, 3) → (15, 3)^float32	numpy.ndarray
        betas               (1, 10) → (10,)^float32	numpy.ndarray
        pred_cam            (1, 3) → (3,)^float32	numpy.ndarray
        pred_keypoints_3d   (1, 21, 3) → (21, 3)^float32	numpy.ndarray
        pred_vertices       (1, 778, 3) → (778, 3)^float32	numpy.ndarray
        pred_cam_t_full     (1, 3) → (3,)^float64	numpy.ndarray
        scaled_focal_length 15859.375	numpy.float64
        pred_keypoints_2d   (1, 21, 2) → (21, 2)^float64	numpy.ndarray
     • dict=3, depth=1
    hand_bbox           [437.0, 360.0, 464.0, 402.0]	list
    is_right            0.0	float
    w